In [10]:
from botocore.config import Config
import boto3

# tell botocore to use v4 signing and your “spain” region
my_config = Config(signature_version="s3v4", region_name="spain")

s3 = boto3.client(
    "s3",
    endpoint_url="http://localhost:9000",
    aws_access_key_id="admin",
    aws_secret_access_key="admin123",
    config=my_config,
)

# list objects under your bucket/folder
resp = s3.list_objects(Bucket="deltalake", Prefix="Exploitation_Zone/")
print([o["Key"] for o in resp.get("Contents", [])])

['Exploitation_Zone/exploitation.duckdb']


In [2]:
import bdm_duck as bd

bd.list_tables()
#rates = bd.table("ecb_exchange_rates").df()
#print(rates)

['eodhd_financial_news',
 'finnhub_financial_report',
 'imf_indicators',
 'ops_db_kyc_metadata',
 'ops_db_pdf_metadata',
 'ops_db_user_profiles',
 'ops_db_user_transactions']

In [21]:
import duckdb
con = duckdb.connect("exploitation.duckdb", read_only=True)
q = """
    SELECT table_name
    FROM information_schema.tables
    """
result = con.sql(q).fetchall()
print(result)

[('ecb_exchange_rates',), ('imf_indicators',)]


In [1]:
import bdm_duck as bd
bd.list_tables()


['ecb_exchange_rates', 'imf_indicators']

In [30]:
import os
import duckdb

# 1) Connect & install/load extensions
con = duckdb.connect()
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
con.execute("INSTALL aws;")
con.execute("LOAD aws;")

# 2) Create or replace an S3 secret for MinIO
access_key = os.getenv("MINIO_ROOT_USER", "admin")
secret_key = os.getenv("MINIO_ROOT_PASSWORD", "admin123")
endpoint = os.getenv("MINIO_ENDPOINT", "localhost:9000")

con.execute(
    f"""
CREATE OR REPLACE SECRET minio_secret (
  TYPE s3,
  PROVIDER config,
  KEY_ID    'admin',
  SECRET    'admin123',
  ENDPOINT  'localhost:9000',
  REGION    'us-east-1',
  URL_STYLE 'path',
  USE_SSL   false,
  SCOPE     's3://deltalake/Exploitation_Zone/'
);
"""
)

# 3) Attach without SECRET clause
con.execute(
    """
ATTACH 's3://deltalake/Exploitation_Zone/exploitation.duckdb'
  AS expo_db;  -- implicitly READ_ONLY on S3
"""
)

# 4) Verify it worked
print("Tables in expo_db:", con.execute("SHOW TABLES FROM expo_db").fetchall())

ParserException: Parser Error: syntax error at or near "FROM"

In [22]:
from botocore.config import Config
import boto3

# Configure boto3 for your MinIO
cfg = Config(signature_version="s3v4", region_name="us-east-1")
s3 = boto3.client(
    "s3", endpoint_url="http://localhost:9000", aws_access_key_id="admin", aws_secret_access_key="admin123", config=cfg
)

# This HEAD will fail with 403 if the bucket is private
try:
    s3.head_object(Bucket="deltalake", Key="Exploitation_Zone/exploitation.duckdb")
    print("HEAD succeeded – object is accessible")
except Exception as e:
    print("HEAD failed:", e)

HEAD succeeded – object is accessible


In [18]:
import os
db_file = "exploitation_test.duckdb"
if os.getenv("DOCKER_ENV", "false") == "true":
    _MINIO_ENDPOINT = "minio:9000"
else:
    _MINIO_ENDPOINT = "localhost:9000"
_BUCKET = os.getenv("MINIO_DATA_BUCKET", "deltalake")
_USER = os.getenv("MINIO_ROOT_USER", "admin")
_PASS = os.getenv("MINIO_ROOT_PASSWORD", "admin123")
_REMOTE_DB = os.getenv(
    "BDM_DUCK_REMOTE",
    f"s3://{_BUCKET}/Exploitation_Zone/exploitation.duckdb",
)

_host = _MINIO_ENDPOINT
_use_ssl = "https" in _MINIO_ENDPOINT.lower()
con = duckdb.connect(db_file)

bucket = "deltalake"
con.sql("INSTALL httpfs;")
con.sql("LOAD httpfs;")
con.execute(
    f"""
        CREATE OR REPLACE SECRET minio_secret (
          TYPE s3,
          PROVIDER config,
          KEY_ID    '{_USER}',
          SECRET    '{_PASS}',
          ENDPOINT  '{_host}',
          REGION    'us-east-1',
          URL_STYLE 'path',
          USE_SSL   {'true' if _use_ssl else 'false'}
        );
        """
)

for name in ["ecb_exchange_rates", "imf_indicators"]:
    con.sql(
        f"""
        CREATE OR REPLACE TABLE {name} AS
        SELECT * FROM parquet_scan(
            's3://{bucket}/_staging_parquet/{name}/*.parquet',
            filename=true, hive_partitioning=true
        );
    """
    )
    result = con.sql(f"SELECT * FROM {name}").df()
    print(result)

con.sql("CHECKPOINT")
con.commit()
con.close()

          time currency        rate  \
0   2021-01-18      AUD      1.5721   
1   2021-01-18      HRK      7.5535   
2   2021-01-18      HUF    360.3900   
3   2021-01-18      IDR  17053.6700   
4   2021-01-18      ILS      3.9018   
..         ...      ...         ...   
635 2021-01-28      USD      1.2091   
636 2021-01-15      USD      1.2123   
637 2021-01-08      USD      1.2250   
638 2021-01-05      USD      1.2271   
639 2021-01-14      USD      1.2124   

                                              filename  
0    s3://deltalake/_staging_parquet/ecb_exchange_r...  
1    s3://deltalake/_staging_parquet/ecb_exchange_r...  
2    s3://deltalake/_staging_parquet/ecb_exchange_r...  
3    s3://deltalake/_staging_parquet/ecb_exchange_r...  
4    s3://deltalake/_staging_parquet/ecb_exchange_r...  
..                                                 ...  
635  s3://deltalake/_staging_parquet/ecb_exchange_r...  
636  s3://deltalake/_staging_parquet/ecb_exchange_r...  
637  s3://deltalak

In [17]:
con = duckdb.connect(db_file)
result = con.sql(f"SELECT * FROM {name}").df()
print(result)

     year      indicator    value  \
0    1993  USA_NGDP_RPCH    2.800   
1    1998  USA_NGDP_RPCH    4.500   
2    1990  USA_NGDP_RPCH    1.900   
3    2023  USA_NGDP_RPCH    2.900   
4    1985  USA_NGDP_RPCH    4.200   
..    ...            ...      ...   
995  2008         USA_LP  304.718   
996  2021         ESP_LP   47.331   
997  1981         USA_LP  229.916   
998  2003         USA_LP  290.733   
999  2021         USA_LP  332.330   

                                              filename  
0    s3://deltalake/_staging_parquet/imf_indicators...  
1    s3://deltalake/_staging_parquet/imf_indicators...  
2    s3://deltalake/_staging_parquet/imf_indicators...  
3    s3://deltalake/_staging_parquet/imf_indicators...  
4    s3://deltalake/_staging_parquet/imf_indicators...  
..                                                 ...  
995  s3://deltalake/_staging_parquet/imf_indicators...  
996  s3://deltalake/_staging_parquet/imf_indicators...  
997  s3://deltalake/_staging_parquet/imf_i